# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [5]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
mashable = pd.read_csv(url, index_col=0)
mashable.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [6]:
mashable.shape

(6000, 61)

In [7]:
X = mashable.drop(['url', 'Popular'], axis=1)
y = mashable['Popular']

In [8]:
y.mean()

0.5

In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(mashable, random_state=1)

In [10]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [11]:
import numpy as np

In [12]:
test1=np.array([X_test, y_test])
test=pd.DataFrame(data=test1)

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor

models = {'lr': LogisticRegression(),
          'dt': DecisionTreeRegressor()}

In [14]:
# hacer el punto fit para cada modelo
for model in models.keys():
    models[model].fit(X_train, y_train)

In [15]:
# predict test for each model
y_pred = pd.DataFrame(index=test_df.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [16]:
y_pred.head()

,lr,dt
1483,1,0.0
2185,1,0.0
2520,1,1.0
3721,1,0.0
3727,0,0.0


In [17]:
y_pred.mean(axis=1).head()

1483    0.5
2185    0.5
2520    1.0
3721    0.5
3727    0.0
dtype: float64

In [18]:
#dt = DecisionTreeRegressor()

In [19]:
#dt.fit(y_pred, y_test)

In [20]:
# Evaluate each model
from sklearn.metrics import mean_squared_error

for model in models.keys():
    print(model,np.sqrt(mean_squared_error(y_pred[model], y_test)))

lr 0.6082762530298219
dt 0.6777413469261953


#### Estimación de Accuracy

In [21]:
from sklearn import metrics
print ('acc_lr:',metrics.accuracy_score(y_pred['lr'], y_test),'\n','acc_dt',metrics.accuracy_score(y_pred['dt'], y_test))

acc_lr: 0.63 
 acc_dt 0.5406666666666666


De acuerdo a los resultados obtenidos, el modelo con un accuracy más alto es la Regresión logística, por encima aproximadamente de un 10%.

#### Estimación de F1-Score

In [22]:
X = mashable.drop(['url', 'Popular'], axis=1)
y = mashable['Popular']

In [23]:
from sklearn import metrics
print ('acc_lr:',metrics.f1_score(y_pred['lr'], y_test),'\n','acc_dt',metrics.f1_score(y_pred['dt'], y_test))

acc_lr: 0.6127006280530356 
 acc_dt 0.5421926910299003


Por medio de la estimación del F1-Score, como otra medida de precisión, se concluye de igual forma que en el ítem anterior, que la estimación del modelo que mejor se comporta es la regresión logística, aunque la diferencia es aproximadamente un 9%. 

# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [24]:
n_estimators = 300
# set a seed for reproducibility
np.random.seed(123)

n_samples = X_train.shape[0]

# create bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]


In [25]:
(samples)[10]

array([4403, 4011, 2930, ..., 2299,  851, 3847])

In [26]:
# show the rows for the first decision tree
X_train.iloc[samples[0], :].head()

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
2239,142.0,11.0,135.0,0.631579,1.0,0.690000,11.0,4.0,8.0,0.0,...,0.354762,0.285714,0.5,-0.308333,-0.625,-0.100000,0.333333,0.250000,0.166667,0.250000
4838,349.0,10.0,1411.0,0.431259,1.0,0.630996,7.0,6.0,4.0,1.0,...,0.438312,0.050000,1.0,-0.322500,-0.800,-0.100000,1.000000,0.100000,0.500000,0.100000
1047,731.0,12.0,238.0,0.621277,1.0,0.823077,3.0,1.0,1.0,0.0,...,0.291259,0.100000,0.6,0.000000,0.000,0.000000,0.000000,0.000000,0.500000,0.000000
373,583.0,8.0,253.0,0.661355,1.0,0.846667,7.0,0.0,0.0,1.0,...,0.432149,0.050000,1.0,-0.247194,-0.500,-0.071429,0.400000,-0.400000,0.100000,0.400000
2595,659.0,15.0,458.0,0.512088,1.0,0.662879,5.0,3.0,1.0,0.0,...,0.368586,0.136364,0.8,-0.219167,-0.500,-0.125000,0.454545,0.136364,0.045455,0.136364


Entrenamiento de los datos teniendo en cuenta los tres sets de clasificaciones.
* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [27]:
treeReg1 = DecisionTreeRegressor(max_depth=None, random_state=123)
treeReg2 = DecisionTreeRegressor(max_depth=2, random_state=123)
logReg   = LogisticRegression(solver='liblinear',C=1e9)
for i in range(n_estimators):
    if i<100:
        treeReg1.fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
    elif i>=100 and i<200:
        treeReg2.fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])
    else:
        logReg.fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [28]:
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    if i<100:
        y_pred_df.iloc[:, i] = treeReg1.predict(X_test)
    elif i>=100 and i<200:
        y_pred_df.iloc[:, i] = treeReg2.predict(X_test)
    else:
        y_pred_df.iloc[:, i] = logReg.predict(X_test)
y_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1483,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,1,1,1,1,1,1,1,1,1
2185,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,1,1,1,1,1,1,1,1,1
2520,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,1,1,1,1,1,1,1,1,1
3721,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1,1,1,1,1,1,1,1,1,1
3727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
y_pred_df.sum(axis=1).head()

1483    274.314721
2185    255.903457
2520    255.903457
3721    255.903457
3727     25.679348
dtype: float64

In [30]:
#y_pred_df.iloc[:,100]

## Calculando el F1-score y accuracy para los tres modelos como si fueran uno

In [31]:
y_pred = (y_pred_df.sum(axis=1) >= (n_estimators / 2)).astype(np.int)

from sklearn import metrics
metrics.f1_score(y_pred, y_test)

0.64

In [32]:
metrics.accuracy_score(y_pred, y_test)

0.64

In [33]:
res=[]
for i in range(y_pred_df.shape[1]):
   res.append (metrics.accuracy_score(y_pred_df.iloc[:,1], y_test))

In [34]:
pd.Series(res).describe()

count    3.000000e+02
mean     5.566667e-01
std      2.780195e-15
min      5.566667e-01
25%      5.566667e-01
50%      5.566667e-01
75%      5.566667e-01
max      5.566667e-01
dtype: float64

## Usando sklearn

### para max_depth=none

In [35]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bagreg = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=None, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1, n_jobs=-1)
bagreg.fit(X_train, y_train)
y_pred1 = bagreg.predict(X_test)
y_pred1

array([1, 1, 1, ..., 1, 0, 0])

In [36]:
metrics.f1_score(y_pred1, y_test), metrics.accuracy_score(y_pred1, y_test)

(0.635752688172043, 0.6386666666666667)

### para max_depth=2

In [37]:
bagreg = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=2, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1, n_jobs=-1)
bagreg.fit(X_train, y_train)
y_pred2 = bagreg.predict(X_test)
y_pred2

array([1, 1, 1, ..., 1, 0, 0])

In [38]:
metrics.f1_score(y_pred2, y_test), metrics.accuracy_score(y_pred2, y_test)

(0.6434782608695653, 0.6446666666666667)

### logistic regression

In [39]:
logReg = BaggingClassifier(base_estimator=LogisticRegression(solver='liblinear',C=1e9, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1, n_jobs=-1)
logReg.fit(X_train, y_train)
y_pred3 = logReg.predict(X_test)
y_pred3

array([1, 1, 1, ..., 1, 0, 0])

In [128]:
metrics.f1_score(y_pred3, y_test), metrics.accuracy_score(y_pred3, y_test)

(0.6085136078157711, 0.626)

En resumen, los resultados de F1-Score y Accuracy para cada modelo son:

max_depth=none   0,6357 y 0,6386

max_depth=2      0,6434 y 0,6446

Log. regression  0,6085 y 0,626

Se concluye que el modelo que ofrece la mejor precisión es el max_depth=2      0,6434 y 0,6446

# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [40]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)

In [41]:
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6456278763971072, 0.6406666666666667)

Utilizando el modelo de Majority voting, si se tienen en cuenta el mismo número de decimales, es indifierente realizar la aproximación por cualquiera de los dos métodos, sin embargo y para efectos de mantener la mayor precisión, el método escogido sería el F1-score.

# Exercise 9.4

Estimate te probability as %models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

Estimando la probabilidad

In [42]:
#y_pred=(clf.predict_proba(X_test)[:,1]>=0.5).astype(bool)
y_pred=clf.predict_proba(X_test)
print(y_pred)

[[0.22 0.78]
 [0.39 0.61]
 [0.42 0.58]
 ...
 [0.37 0.63]
 [0.65 0.35]
 [0.72 0.28]]


Cambiando el threshold

In [43]:
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    y_pred=(clf.predict_proba(X_test)[:,1]>=i).astype(bool)
    print('threshold:',i,'  ','f1:',metrics.f1_score(y_pred, y_test),'  ','acc:', metrics.accuracy_score(y_pred, y_test))

threshold: 0.1    f1: 0.674709562109026    acc: 0.5146666666666667
threshold: 0.2    f1: 0.6829493087557604    acc: 0.5413333333333333
threshold: 0.3    f1: 0.6986166007905139    acc: 0.5933333333333334
threshold: 0.4    f1: 0.6990715456034954    acc: 0.6326666666666667
threshold: 0.5    f1: 0.6558861578266494    acc: 0.6453333333333333
threshold: 0.6    f1: 0.5405844155844156    acc: 0.6226666666666667
threshold: 0.7    f1: 0.3350357507660879    acc: 0.566
threshold: 0.8    f1: 0.12832929782082322    acc: 0.52
threshold: 0.9    f1: 0.015645371577574965    acc: 0.49666666666666665


Luego de cambiar el threshold, en el de 0.4 y 0.5 es donde se obtienen los valores máximos tanto para el f1-score como para el accuracy.

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [44]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6456278763971072, 0.6406666666666667)

In [45]:
errors = np.zeros(clf.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], clf.n_estimators))

for i in range(clf.n_estimators):
    oob_sample = ~clf.estimators_samples_[i]
    y_pred_ = clf.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = clf.estimators_[i].predict(X_test)
    
alpha = (1 - errors) / (1 - errors).sum()
y_pred = (np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int)

In [46]:
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6497064579256361, 0.642)

Usando el oob se observa que el f1 score y el accuracy aumento, aunque el cambio no fue significativo en comparación al majority voting

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [47]:
for i in range(clf.n_estimators):
    oob_sample = ~clf.estimators_samples_[i]
    for j in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        y_pred_ = (clf.estimators_[i].predict_proba(X_train.values[oob_sample])[:,1]>=j).astype(bool)
        errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
        y_pred_all_[:, i] = (clf.estimators_[i].predict_proba(X_test)[:,1]>=j).astype(bool)

alpha = (1 - errors) / (1 - errors).sum()
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    y_pred = (np.sum(y_pred_all_ * alpha, axis=1) >= i).astype(np.int)
    print('threshold:',i,'  ','f1:',metrics.f1_score(y_pred, y_test),'  ','acc:', metrics.accuracy_score(y_pred, y_test))

threshold: 0.1    f1: 0.6747203579418345    acc: 0.5153333333333333
threshold: 0.2    f1: 0.683894785417628    acc: 0.5433333333333333
threshold: 0.3    f1: 0.6987593052109181    acc: 0.5953333333333334
threshold: 0.4    f1: 0.7012700165654334    acc: 0.6393333333333333
threshold: 0.5    f1: 0.6497064579256361    acc: 0.642
threshold: 0.6    f1: 0.5341002465078061    acc: 0.622
threshold: 0.7    f1: 0.32057911065149947    acc: 0.562
threshold: 0.8    f1: 0.1240875912408759    acc: 0.52
threshold: 0.9    f1: 0.015665796344647518    acc: 0.49733333333333335


El threshold 0.4 es el punto donde los dos valores alcanzan su valor màximo. De otro lado, en el 0.9 es donde tanto el f1-score como el acurracy toman su valor mínimo.

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score

In [48]:
from sklearn.linear_model import LogisticRegressionCV
y_pred_all_ = np.zeros((X_test.shape[0], clf.n_estimators))
X_train_2 = np.zeros((X_train.shape[0], clf.n_estimators))

for i in range(clf.n_estimators):

    X_train_2[:, i] = clf.estimators_[i].predict(X_train)
    y_pred_all_[:, i] = clf.estimators_[i].predict(X_test)
    
lr = LogisticRegressionCV(cv=5)
lr.fit(X_train_2, y_train)

y_pred = lr.predict(y_pred_all_)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6474064346684175, 0.642)

cambiando el threshold

In [49]:
for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    y_pred=(lr.predict_proba(y_pred_all_)[:,1]>=i).astype(bool)
    print('threshold:',i,'  ','f1:',metrics.f1_score(y_pred, y_test),'  ','acc:', metrics.accuracy_score(y_pred, y_test))

threshold: 0.1    f1: 0.6702127659574468    acc: 0.504
threshold: 0.2    f1: 0.6744290192566056    acc: 0.5153333333333333
threshold: 0.3    f1: 0.6956521739130436    acc: 0.5753333333333334
threshold: 0.4    f1: 0.6968716289104638    acc: 0.6253333333333333
threshold: 0.5    f1: 0.6474064346684175    acc: 0.642
threshold: 0.6    f1: 0.5029635901778153    acc: 0.6086666666666667
threshold: 0.7    f1: 0.2399103139013453    acc: 0.548
threshold: 0.8    f1: 0.015645371577574965    acc: 0.49666666666666665
threshold: 0.9    f1: 0.0    acc: 0.496


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


La probabilidad que maximiza el valor de fq-score y el accuracy está entre 0.4 y 0.5.